In [ ]:
import copy
import cv2
import os
import rootutils
import numpy as np
import open3d as o3d

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.utils import remove_small_clusters, crop_by_height, align_pointclouds, compute_pcd_differences

#### Read pcd

In [ ]:
target_filename = 'left_base_1'
target_pcd = o3d.io.read_point_cloud(f'../output/pcd/anomalies/{target_filename}.ply')
target_left_img = cv2.imread(f'../data/left_anomaly_2/{target_filename}.png')

source_filename = 'left_20250917_154339_3'
source_pcd = o3d.io.read_point_cloud(f'../output/pcd/anomalies/{source_filename}.ply')
source_left_img = cv2.imread(f'../data/left_anomaly_2/{source_filename}.png')

#### Preprocess

In [ ]:
transformation = np.load('../camera_configs/transformation_matrix.npy')

In [ ]:
target_pcd = remove_small_clusters(target_pcd, voxel_size=0.01, min_cluster_size=200)
source_pcd = remove_small_clusters(source_pcd, voxel_size=0.01, min_cluster_size=200)

In [ ]:
target_pcd = target_pcd.transform(transformation)
source_pcd = source_pcd.transform(transformation)

#### Crop

In [ ]:
source_copy = copy.deepcopy(source_pcd)
target_copy = copy.deepcopy(target_pcd)
source_copy.paint_uniform_color([1, 0, 0])  # Red for source
target_copy.paint_uniform_color([0, 1, 0])  # Green for target

In [ ]:
source_copy = crop_by_height(source_copy, z_min=1, z_max=2)
target_copy = crop_by_height(target_copy, z_min=1, z_max=2)
o3d.visualization.draw_geometries([source_copy, target_copy], window_name="Preprocessed Pointclouds")

#### Align

In [ ]:
# Align pointclouds
aligned_source, transformation = align_pointclouds(source_copy, target_copy)
print("Alignment complete. Transformation matrix:")
print(transformation)

#### Compute differences

In [ ]:
distance_pcd = compute_pcd_differences(aligned_source, target_copy, max_dist=0.01)
o3d.visualization.draw_geometries([distance_pcd], window_name="Anomaly Map")